<a href="https://colab.research.google.com/github/CyanideBoy/Mantra_demo/blob/master/eval_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!rm -rf Mantra_demo
!git clone https://github.com/CyanideBoy/Mantra_demo.git

Cloning into 'Mantra_demo'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 563 (delta 3), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (563/563), 136.69 MiB | 32.72 MiB/s, done.
Resolving deltas: 100% (219/219), done.
Checking out files: 100% (516/516), done.


In [2]:
import os
import sys

sys.path.insert(0,'Mantra_demo')

import numpy as np 
import cv2 as cv2
import lmdb
import keras
from matplotlib import pyplot
np.set_printoptions( 3, suppress = True )
from tensorflow.python.client import device_lib
from tf_multi_gpu import make_parallel 
print device_lib.list_local_devices()
from keras.utils import to_categorical
import modelCore

print cv2.__version__
fontsize = 24

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1078140188034994020
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7824641803535032092
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13625588030615240786
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2548465663738854049
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11375751988
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 4623123162346270461
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:02:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
m

In [3]:
debug = False   # Setting it True gives performance similar to what paper reports
idx = 1         # Model id (0-4)

# Prepare Model and Metrics

In [4]:
#import sys
#sys.path.insert( 0, 'sequence/')

manTraNet_root = './Mantra_demo'
manTraNet_modelDir = os.path.join( manTraNet_root, 'pretrained_weights' )

def get_single_gpu_model(idx) :
    
    mantra_model = modelCore.load_pretrain_model_by_index( idx, manTraNet_modelDir )
    print mantra_model.summary(line_length=120)
    
    return mantra_model


# Metrics

In [5]:
from sklearn.metrics import roc_auc_score, f1_score
import tensorflow as tf

def np_F1( y_true, y_pred ) :
    score = []
    for yy_true, yy_pred in zip( y_true, y_pred ) :
        this = f1_score( (yy_true>.5).astype('int').ravel(), (yy_pred>.5).astype('int').ravel() )
        that = f1_score( (yy_true>.5).astype('int').ravel(), (1-yy_pred>.5).astype('int').ravel() )
        score.append( max( this, that ) )
    return np.mean( score ).astype('float32')


def F1( y_true, y_pred ) :
    return tf.py_func( np_F1, [y_true, y_pred], 'float32')

def np_auc( y_true, y_pred ) :
    score = []

    for yy_true, yy_pred in zip( y_true, y_pred ) :
        this = roc_auc_score( (yy_true>.5).astype('int').ravel(), yy_pred.ravel() )
        that = roc_auc_score( (yy_true>.5).astype('int').ravel(), 1-yy_pred.ravel() )
        score.append( max( this, that ) )
    return np.mean( score ).astype('float32')

def auroc(y_true, y_pred):
    return tf.py_func( np_auc, [y_true, y_pred], 'float32')

# Prepare Dataset

In [19]:
def prepare_coverage_dataset() :
    input_image_file_list = 'COVERAGE_image_new.list'
    #input_image_file_list = os.path.join(manTraNet_root, 'VERAGE')
    with open( 'Mantra_demo/' + input_image_file_list, 'r') as IN :
        input_files = [ 'Mantra_demo/'+line.strip() for line in IN.readlines() ]
    print "INFO: successfully load", len( input_files ), "input files"

    def get_input_ID( input_file ) :
        bname = os.path.basename( input_file )
        return bname.rsplit('.')[0]

    def get_mask_file_from_ID( sample_id ) :
        return os.path.join('Mantra_demo/VERAGE/mask/', '{}forged.tif'.format(sample_id[:-1]) ) 

    def preprocess( input_image, input_mask ) :
        x = np.expand_dims( input_image, axis=0 ).astype('float32')/255. * 2 - 1
        y = np.expand_dims( np.expand_dims( input_mask, axis=0 ), axis=-1 )/255.
        return x, y

    raw_lut = dict( zip( [ get_input_ID(f) for f in input_files ], input_files) )

    paired_results = []
    for key in raw_lut.keys() : 
        raw_file = raw_lut[key]
        mask_file = get_mask_file_from_ID(key)
        
        r = cv2.imread(raw_file, 1 )[...,::-1]
        m = cv2.imread(mask_file, 0)
        if r.shape[:2] != m.shape[:2] :
            continue
        
        raw_mask_dec = (raw_file, mask_file )
        paired_results.append( raw_mask_dec )

    print len(paired_results)
    return paired_results, len(paired_results), preprocess

# Evaluate model performance for each dataset

In [7]:
def create_evaluation_data_generator( paired_results, preprocess ) :
    for raw_file, mask_file in paired_results :
        r = cv2.imread( raw_file, 1 )[...,::-1]
        m = cv2.imread( mask_file, 0)
        if r.shape[:2] != m.shape[:2] :
            print "INFO: find unmatched", raw_file, mask_file, ", skip"
            continue
        x, y = preprocess( r, m )
        yield x, y

# Result

In [20]:
model = get_single_gpu_model(idx)
model.compile( optimizer='sgd',
               loss ='binary_crossentropy',
               metrics=['accuracy', F1, auroc], )


from prettytable import PrettyTable
import os
import json

table = PrettyTable()
table.field_names = ['Dataset', 'Loss', 'Acc/AUC', 'F1' ]

mega_lut = dict()

for prepare_dataset, name in zip( [ prepare_coverage_dataset ],
                                  ['COVERAGE'] ) :
    input_pairs, L, preprocess = prepare_dataset()
    # create data generator
    datagen = create_evaluation_data_generator( input_pairs, preprocess ) 
    res = model.evaluate_generator( datagen, L if not debug else 1, verbose=1 )
    # print 
    lut = dict( zip( model.metrics_names, res ) )
    print name, lut
    mega_lut[name] = lut
    # update
    table.add_row( [name] + [ "{:.4f}".format(lut[key]) for key in ['loss', 'auroc', 'F1'] ] )

print (table)


INFO: use activation in the last CONV=None
INFO: unfreeze feature extraction part, trainable=True
('INFO: freeze', 'image_in')
('INFO: freeze', 'b1c1')
('INFO: freeze', 'b1c2')
('INFO: freeze', 'b2c1')
('INFO: freeze', 'b2c2')
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
img_in (InputLayer)                    (None, None, None, 3)      0                                                     
________________________________________________________________________________________________________________________
Featex (Model)                         (None, None, None, 256)    3680989       img_in[0][0]                            
________________________________________________________________________________________________________________________
outlierTrans (Conv2D)                  (None, N